# 🚀 Planning & Design Agent Example

This notebook demonstrates how to create intelligent planning agents that can break down complex tasks and coordinate multiple specialized agents.

## 💻 Windows ARM64 Setup Instructions

If you're running on Windows ARM64, you may need to configure OpenSSL:

```bash
git config --global core.longpaths true
winget install ShiningLight.OpenSSL.Dev
$env:OPENSSL_DIR="C:\Program Files\OpenSSL-Win64-ARM"
$env:OPENSSL_LIB_DIR="C:\Program Files\OpenSSL-Win64-ARM\lib\VC\arm64\MT"
$env:OPENSSL_STATIC="1"
```

In [1]:
# 🧹 Clean Up Previous Installations (Optional)
# Uncomment these lines if you need to remove previous agent framework versions
# This ensures a clean installation of the latest version
# ! pip uninstall agent-framework -y
# ! pip uninstall agent-framework-azure -y
# ! pip uninstall agent-framework-foundry -y

In [2]:
# 📦 Install Required Packages (Optional)
# Uncomment this line to install all required dependencies
# The -U flag ensures packages are upgraded to latest versions
# ! pip install -r requirements.txt --constraint constraints.txt -U

In [3]:
# 🔌 Install Additional Dependencies (Optional)
# Uncomment to install WebSocket support and OpenTelemetry for monitoring
# websockets: For real-time communication
# opentelemetry-semantic-conventions-ai: For AI observability and monitoring
# ! pip install websockets opentelemetry-semantic-conventions-ai

In [4]:
# 📦 Import Standard Libraries
# os: For environment variable access
# dotenv: For loading environment variables from .env file
import os

from dotenv import load_dotenv

In [5]:
# 📝 Import Data Modeling Libraries
# Pydantic: For data validation and structured outputs
# BaseModel: Base class for creating data models
# Field: For adding metadata and validation to model fields
# List: Type hint for list collections
from pydantic import BaseModel, Field
from typing import List

In [6]:
# 🤖 Import Microsoft Agent Framework Components
# ChatMessage: For creating structured chat messages
# Role: Enum for message roles (user, assistant, system)
# ChatOptions: Configuration options for chat interactions
# OpenAIChatClient: Client for OpenAI-compatible APIs (GitHub Models)
from agent_framework import ChatMessage, Role, ChatOptions
from agent_framework.openai import OpenAIChatClient

In [7]:
# 🔧 Load Environment Variables
# Load configuration from .env file
# Required: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
load_dotenv()

True

In [8]:
# 🎯 Agent Configuration
# Define the agent's name and role

AGENT_NAME = "TravelAgent"

# 📋 System Instructions for Planning Agent
# This agent acts as a coordinator that decides which specialized agents to use
AGENT_INSTRUCTIONS = """You are a planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialized in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
"""


In [9]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [10]:
options = ChatOptions(response_format=TravelPlan)

In [11]:
client = OpenAIChatClient(base_url=os.environ.get("GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), ai_model_id=os.environ.get("GITHUB_MODEL_ID"))

In [12]:
agent =  client.create_agent(name= AGENT_NAME , instructions=AGENT_INSTRUCTIONS)

In [13]:
messages = [
        ChatMessage(role=Role.USER, text="Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne")
    ]

In [14]:
response = await agent.run(messages,response_format=TravelPlan)

In [15]:
response.messages[0].text

'{"main_task":"Create a travel plan for a family of 4 from Singapore to Melbourne","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Book round-trip flights for a family of 4 from Singapore to Melbourne."},{"assigned_agent":"HotelBooking","task_details":"Find and book a family-friendly hotel in Melbourne that accommodates 4 people."},{"assigned_agent":"CarRental","task_details":"Book a rental car suitable for a family of 4 in Melbourne."},{"assigned_agent":"ActivitiesBooking","task_details":"Suggest and book family-friendly activities in Melbourne."}]}'